# Prototipagem CNN para Classificação de Desfecho de Petições Jurídicas

Este notebook implementa uma **Rede Neural Convolucional Profunda (Deep CNN)** para classificação do resultado/desfecho provável de petições jurídicas.

## Objetivo
Prever o desfecho de uma petição (Deferida, Indeferida, Parcialmente Deferida) baseado no conteúdo textual e metadados da petição.

## Arquitetura
- **Entrada**: Texto processado (embeddings word2vec/GloVe)
- **Camadas Convolucionais 1D**: Extração de padrões textuais locais
- **Pooling**: Redução de dimensionalidade
- **Regularização**: Dropout + BatchNormalization
- **Camadas Densas**: Classificação final

## 1. Importar Bibliotecas Necessárias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sqlite3
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Input, concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Processamento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re

# Métricas e validação
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve, auc
)

# Plotagem e visualização
from sklearn.metrics import ConfusionMatrixDisplay

# Download de recursos NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("✓ Todas as bibliotecas importadas com sucesso!")
print(f"✓ TensorFlow versão: {tf.__version__}")
print(f"✓ GPU disponível: {tf.config.list_physical_devices('GPU')}")

## 2. Carregar e Explorar os Dados

In [ ]:
# Caminho do banco de dados SQLite
DB_PATH = "/gdrive/MyDrive/Colab Notebooks/JFRN/split_dados_01.sqlite3"  # Ajuste o caminho conforme necessário
TABLE_NAME = "peticoes"  # Ajuste o nome da tabela

# Conectar ao banco e carregar dados
try:
    conn = sqlite3.connect(DB_PATH)
    query = f"SELECT * FROM {TABLE_NAME} LIMIT 1000"  # Limite inicial para prototipagem
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"✓ Dados carregados com sucesso!")
    print(f"  - Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
except Exception as e:
    print(f"✗ Erro ao carregar dados: {e}")
    print("\nCriando dataset de exemplo para demonstração...")
    
    # Dataset de exemplo para demonstração
    np.random.seed(42)
    n_samples = 1000
    
    example_texts = [
        "A petição solicita a revisão da sentença anterior com base em novos documentos.",
        "Recurso de apelação contra a decisão de primeira instância.",
        "Moção para reconsideração do pedido inicial.",
        "Ação ordinária para cobrança de valores contratados.",
        "Demanda por responsabilidade civil.",
        "Petição para anulação de ato administrativo.",
        "Recurso extraordinário questionando constitucionalidade.",
        "Processo trabalhista por rescisão contratual.",
    ]
    
    texts = [np.random.choice(example_texts) for _ in range(n_samples)]
    outcomes = np.random.choice(['Deferida', 'Indeferida', 'Parcialmente_Deferida'], n_samples)
    
    df = pd.DataFrame({
        'texto_peticao': texts,
        'desfecho': outcomes,
        'data': pd.date_range('2020-01-01', periods=n_samples, freq='D'),
        'valor': np.random.uniform(1000, 100000, n_samples)
    })
    
    print(f"✓ Dataset de exemplo criado: {df.shape[0]} linhas x {df.shape[1]} colunas")

# Exibir informações do dataset
print("\n" + "="*80)
print("INFORMAÇÕES DO DATASET")
print("="*80)
print(df.head(10))
print(f"\nTipos de dados:\n{df.dtypes}")
print(f"\nValores nulos:\n{df.isnull().sum()}")

In [ ]:
# Análise exploratória da variável alvo (desfecho)
print("\n" + "="*80)
print("ANÁLISE DA VARIÁVEL ALVO: DESFECHO")
print("="*80)

# Coluna de desfecho (ajuste conforme necessário)
target_column = 'desfecho'  # ou 'resultado', 'outcome', etc.

# Verificar se a coluna existe
if target_column not in df.columns:
    print(f"⚠ Coluna '{target_column}' não encontrada.")
    print(f"Colunas disponíveis: {df.columns.tolist()}")
    # Tentar encontrar coluna similar
    possible_cols = [col for col in df.columns if 'result' in col.lower() or 'desfecho' in col.lower() or 'outcome' in col.lower()]
    if possible_cols:
        target_column = possible_cols[0]
        print(f"Usando coluna: {target_column}")
else:
    print(f"✓ Usando coluna: {target_column}")

# Distribuição das classes
class_distribution = df[target_column].value_counts()
print(f"\nDistribuição de classes:")
print(class_distribution)
print(f"\nPercentual:")
print((df[target_column].value_counts(normalize=True) * 100).round(2))

# Visualizar distribuição
plt.figure(figsize=(10, 5))
class_distribution.plot(kind='bar', edgecolor='black', alpha=0.7)
plt.title('Distribuição de Desfechos nas Petições', fontsize=14, fontweight='bold')
plt.xlabel('Desfecho')
plt.ylabel('Frequência')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Verificar desbalanceamento
print(f"\nRazão maior/menor classe: {class_distribution.max() / class_distribution.min():.2f}:1")

## 3. Pré-processamento e Normalização de Texto

In [ ]:
# Função para pré-processamento de texto
def preprocess_text(text):
    """
    Pré-processar texto para treinamento de CNN
    """
    if pd.isna(text):
        return ""
    
    # Converter para string e lowercase
    text = str(text).lower()
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remover emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Remover pontuação especial
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Tokenização
    tokens = word_tokenize(text, language='portuguese')
    
    # Remover stopwords português
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Stemming
    stemmer = SnowballStemmer('portuguese')
    tokens = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(tokens)

# Aplicar pré-processamento
print("Pré-processando textos...")
text_column = 'texto_peticao'  # Ajuste conforme necessário

if text_column not in df.columns:
    possible_cols = [col for col in df.columns if 'text' in col.lower() or 'peticao' in col.lower()]
    if possible_cols:
        text_column = possible_cols[0]
    else:
        raise ValueError(f"Coluna de texto não encontrada. Colunas disponíveis: {df.columns.tolist()}")

df['texto_processado'] = df[text_column].apply(preprocess_text)

print("✓ Textos pré-processados!")
print(f"\nExemplo antes: {df[text_column].iloc[0][:100]}...")
print(f"Exemplo depois: {df['texto_processado'].iloc[0][:100]}...")

# Estatísticas de comprimento
print("\n" + "="*80)
print("ESTATÍSTICAS DE COMPRIMENTO DOS TEXTOS")
print("="*80)

text_lengths = df['texto_processado'].str.split().str.len()
print(f"Comprimento médio: {text_lengths.mean():.2f} palavras")
print(f"Comprimento mínimo: {text_lengths.min()}")
print(f"Comprimento máximo: {text_lengths.max()}")
print(f"Mediana: {text_lengths.median():.2f}")
print(f"Desvio padrão: {text_lengths.std():.2f}")

# Visualizar distribuição de comprimentos
plt.figure(figsize=(12, 5))
plt.hist(text_lengths, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Número de Palavras')
plt.ylabel('Frequência')
plt.title('Distribuição do Comprimento dos Textos Pré-processados')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Tokenização e sequência de palavras
VOCAB_SIZE = 5000  # Tamanho do vocabulário
MAX_SEQUENCE_LENGTH = 200  # Comprimento máximo das sequências

print("\n" + "="*80)
print("TOKENIZAÇÃO E SEQUÊNCIAS")
print("="*80)

# Criar tokenizer
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(df['texto_processado'])

# Converter textos para sequências
sequences = tokenizer.texts_to_sequences(df['texto_processado'])

# Fazer padding
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print(f"✓ Tokenização concluída!")
print(f"  - Tamanho do vocabulário: {len(tokenizer.word_index)}")
print(f"  - Sequências criadas: {X.shape}")
print(f"  - Comprimento máximo: {MAX_SEQUENCE_LENGTH}")

# Codificar labels (desfecho)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[target_column])
y_labels = label_encoder.classes_

print(f"\n✓ Variável alvo codificada!")
print(f"  - Classes: {y_labels}")
print(f"  - Codificação: {dict(zip(y_labels, label_encoder.transform(y_labels)))}")

# Dividir dados em train/validation/test
print("\n" + "="*80)
print("DIVISÃO TRAIN/VALIDATION/TEST")
print("="*80)

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.15, random_state=42, stratify=y_temp
)

print(f"Conjunto de treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Conjunto de validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Conjunto de teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")

# Converter para one-hot encoding
num_classes = len(y_labels)
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_val_cat = keras.utils.to_categorical(y_val, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

print(f"\nShape dos labels (one-hot): {y_train_cat.shape}")

## 4. Definir Arquitetura da Rede Convolucional Profunda

### Arquitetura CNN 1D para Classificação de Texto
- **Embedding Layer**: Converte índices de palavras em vetores densos
- **Múltiplas Camadas Convolucionais 1D**: Com diferentes tamanhos de filtro (3, 4, 5)
- **Batch Normalization**: Normalização entre camadas para estabilidade
- **Global Max Pooling**: Redução de dimensionalidade
- **Dropout**: Regularização para evitar overfitting
- **Camadas Densas**: Classificação final com ativação softmax

In [ ]:
def build_deep_cnn_model(vocab_size, max_length, num_classes, embedding_dim=128):
    """
    Construir modelo CNN profundo para classificação de texto
    
    Parâmetros:
    - vocab_size: Tamanho do vocabulário
    - max_length: Comprimento máximo das sequências
    - num_classes: Número de classes
    - embedding_dim: Dimensionalidade do embedding
    """
    
    # Camada de entrada
    input_layer = Input(shape=(max_length,))
    
    # Embedding layer
    embedding = Embedding(vocab_size, embedding_dim, input_length=max_length)(input_layer)
    
    # Múltiplas camadas convolucionais com diferentes tamanhos de filtro
    filter_sizes = [3, 4, 5, 7]
    num_filters = 100
    conv_layers = []
    
    for filter_size in filter_sizes:
        conv = Conv1D(
            filters=num_filters,
            kernel_size=filter_size,
            activation='relu',
            kernel_regularizer=keras.regularizers.l2(0.001)
        )(embedding)
        
        # Batch Normalization
        conv = BatchNormalization()(conv)
        
        # Max Pooling
        conv = MaxPooling1D(pool_size=2)(conv)
        
        # Dropout
        conv = Dropout(0.3)(conv)
        
        # Global Max Pooling
        conv = GlobalAveragePooling1D()(conv)
        
        conv_layers.append(conv)
    
    # Concatenar outputs das camadas convolucionais
    merged = concatenate(conv_layers) if len(conv_layers) > 1 else conv_layers[0]
    
    # Camadas densas
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(merged)
    dense = BatchNormalization()(dense)
    dense = Dropout(0.4)(dense)
    
    dense = Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense)
    dense = BatchNormalization()(dense)
    dense = Dropout(0.3)(dense)
    
    dense = Dense(64, activation='relu')(dense)
    dense = Dropout(0.2)(dense)
    
    # Camada de saída
    output = Dense(num_classes, activation='softmax')(dense)
    
    # Criar modelo
    model = models.Model(inputs=input_layer, outputs=output)
    
    return model

# Construir modelo
print("\n" + "="*80)
print("CONSTRUINDO ARQUITETURA CNN PROFUNDA")
print("="*80)

EMBEDDING_DIM = 128

model = build_deep_cnn_model(
    vocab_size=VOCAB_SIZE,
    max_length=MAX_SEQUENCE_LENGTH,
    num_classes=num_classes,
    embedding_dim=EMBEDDING_DIM
)

# Exibir resumo da arquitetura
print("\n✓ Modelo construído com sucesso!")
print("\nArquitetura do Modelo:")
print("-" * 80)
model.summary()

## 5. Compilar o Modelo

In [ ]:
print("\n" + "="*80)
print("COMPILANDO MODELO")
print("="*80)

# Configurar otimizador e função de perda
optimizer = Adam(learning_rate=0.001)
loss_fn = 'categorical_crossentropy'
metrics_list = ['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]

# Compilar modelo
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=metrics_list
)

print("\n✓ Modelo compilado com sucesso!")
print(f"  - Otimizador: Adam (lr=0.001)")
print(f"  - Função de perda: {loss_fn}")
print(f"  - Métricas: {[m if isinstance(m, str) else m.__class__.__name__ for m in metrics_list]}")

## 6. Treinar o Modelo

In [ ]:
print("\n" + "="*80)
print("TREINANDO MODELO")
print("="*80)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Configurações de treinamento
BATCH_SIZE = 32
EPOCHS = 50

# Treinar modelo
print(f"\nParâmetros:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Épocas: {EPOCHS}")
print(f"  - Amostras de treino: {X_train.shape[0]}")
print(f"  - Amostras de validação: {X_val.shape[0]}")

history = model.fit(
    X_train, y_train_cat,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Treinamento concluído!")
print(f"  - Épocas executadas: {len(history.history['loss'])}")
print(f"  - Melhor val_loss: {min(history.history['val_loss']):.4f}")
print(f"  - Melhor val_accuracy: {max(history.history['val_accuracy']):.4f}")

## 7. Avaliar o Desempenho do Modelo

In [ ]:
print("\n" + "="*80)
print("AVALIAÇÃO DO MODELO NO CONJUNTO DE TESTE")
print("="*80)

# Avaliar no conjunto de teste
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(
    X_test, y_test_cat, verbose=0
)

print(f"\nMétricas no Conjunto de Teste:")
print(f"  - Loss: {test_loss:.4f}")
print(f"  - Accuracy: {test_accuracy:.4f}")
print(f"  - Precision: {test_precision:.4f}")
print(f"  - Recall: {test_recall:.4f}")
print(f"  - F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

# Fazer predições
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Relatório de classificação
print("\n" + "="*80)
print("RELATÓRIO DE CLASSIFICAÇÃO DETALHADO")
print("="*80)
print(classification_report(y_test, y_pred, target_names=y_labels, digits=4))

# Matriz de confusão
print("\n" + "="*80)
print("MATRIZ DE CONFUSÃO")
print("="*80)

cm = confusion_matrix(y_test, y_pred)
print("\n", cm)

# Calcular F1-Score por classe
f1_scores = {}
for i, label in enumerate(y_labels):
    f1 = f1_score(y_test == i, y_pred == i, average='binary')
    f1_scores[label] = f1

print("\nF1-Score por classe:")
for label, f1 in f1_scores.items():
    print(f"  - {label}: {f1:.4f}")

## 8. Realizar Predições em Novos Dados

In [ ]:
def predict_petition_outcome(text, model, tokenizer, label_encoder, max_length=MAX_SEQUENCE_LENGTH):
    """
    Fazer predição de desfecho para uma nova petição
    """
    # Pré-processar texto
    processed_text = preprocess_text(text)
    
    # Converter para sequência
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    
    # Fazer predição
    prediction_proba = model.predict(padded, verbose=0)[0]
    predicted_class = np.argmax(prediction_proba)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return {
        'texto': text[:100] + '...' if len(text) > 100 else text,
        'desfecho_predito': predicted_label,
        'confianca': prediction_proba[predicted_class],
        'probabilidades': {label: float(prob) for label, prob in zip(label_encoder.classes_, prediction_proba)}
    }

print("\n" + "="*80)
print("PREDIÇÕES EM NOVOS DADOS")
print("="*80)

# Testar com exemplos do conjunto de teste
print("\nExemplos de predições do conjunto de teste:\n")

for i in range(min(5, len(X_test))):
    texto_original = df[text_column].iloc[-len(X_test) + i]
    resultado_real = y_labels[y_test[i]]
    resultado_predito = y_labels[y_pred[i]]
    confianca = y_pred_proba[i][y_pred[i]]
    
    print(f"Exemplo {i+1}:")
    print(f"  Texto: {texto_original[:80]}...")
    print(f"  Resultado real: {resultado_real}")
    print(f"  Resultado predito: {resultado_predito}")
    print(f"  Confiança: {confianca:.4f}")
    print(f"  Acerto: {'✓' if resultado_real == resultado_predito else '✗'}")
    print()

# Testar com novos textos de exemplo
print("\nPredições em novos textos de exemplo:\n")

novos_textos = [
    "Recurso extraordinário fundamentado em violação de direito constitucional e jurisprudência consolidada",
    "Petição inicial requerendo condenação por responsabilidade civil e danos morais",
    "Moção para reconsideração apresentando novas provas documentais"
]

for texto in novos_textos:
    resultado = predict_petition_outcome(texto, model, tokenizer, label_encoder)
    print(f"Texto: {resultado['texto']}")
    print(f"Desfecho predito: {resultado['desfecho_predito']}")
    print(f"Confiança: {resultado['confianca']:.4f}")
    print(f"Probabilidades por classe:")
    for classe, prob in resultado['probabilidades'].items():
        print(f"  - {classe}: {prob:.4f}")
    print()

## 9. Visualizar Resultados e Métricas

In [ ]:
print("\n" + "="*80)
print("VISUALIZAÇÕES DE RESULTADOS")
print("="*80)

# 1. Histórico de treinamento (Loss)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Histórico de Loss durante Treinamento', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('Acurácia', fontsize=12)
axes[1].set_title('Histórico de Acurácia durante Treinamento', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Matriz de Confusão
fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=y_labels)
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Matriz de Confusão - Conjunto de Teste', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 3. Métricas por classe
metrics_by_class = {}
for i, label in enumerate(y_labels):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    fn = cm[i, :].sum() - cm[i, i]
    tp = cm[i, i]
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    metrics_by_class[label] = {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'support': (y_test == i).sum()
    }

# Plotar métricas por classe
metrics_df = pd.DataFrame(metrics_by_class).T
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(y_labels))
width = 0.25

ax.bar(x - width, metrics_df['precision'], width, label='Precisão', alpha=0.8)
ax.bar(x, metrics_df['recall'], width, label='Recall', alpha=0.8)
ax.bar(x + width, metrics_df['f1'], width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Classe', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Métricas de Desempenho por Classe', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(y_labels)
ax.legend(fontsize=10)
ax.set_ylim([0, 1.1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Visualizações geradas com sucesso!")

In [ ]:
print("\n" + "="*80)
print("VISUALIZAÇÕES ADICIONAIS")
print("="*80)

# 4. Distribuição de probabilidades preditas
fig, axes = plt.subplots(1, len(y_labels), figsize=(15, 5))

for idx, label in enumerate(y_labels):
    class_probs = y_pred_proba[y_test == idx, idx]
    axes[idx].hist(class_probs, bins=30, edgecolor='black', alpha=0.7, color=f'C{idx}')
    axes[idx].set_xlabel('Probabilidade', fontsize=11)
    axes[idx].set_ylabel('Frequência', fontsize=11)
    axes[idx].set_title(f'Confiança - {label}', fontsize=12, fontweight='bold')
    axes[idx].set_xlim([0, 1])
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# 5. Curva ROC (One-vs-Rest) para classificação multiclasse
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

y_test_bin = label_binarize(y_test, classes=range(num_classes))

fig, ax = plt.subplots(figsize=(10, 8))

colors = plt.cm.Set1(np.linspace(0, 1, num_classes))
for i, label in enumerate(y_labels):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, color=colors[i], lw=2, label=f'{label} (AUC = {roc_auc:.3f})')

ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Classificador Aleatório')
ax.set_xlabel('Taxa de Falso Positivo', fontsize=12)
ax.set_ylabel('Taxa de Verdadeiro Positivo', fontsize=12)
ax.set_title('Curvas ROC - One-vs-Rest', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 6. Exemplos de predições corretas e incorretas
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
fig.suptitle('Exemplos de Predições: Corretas (superior) e Incorretas (inferior)', 
             fontsize=14, fontweight='bold')

# Corretas
correct_indices = np.where(y_pred == y_test)[0]
for i in range(min(3, len(correct_indices))):
    idx = correct_indices[i]
    ax = axes[0, i]
    
    # Criar visualização de confiança
    classes = y_labels
    probs = y_pred_proba[idx]
    ax.barh(classes, probs, color=['green' if p == max(probs) else 'lightgray' for p in probs])
    ax.set_xlim([0, 1])
    ax.set_xlabel('Probabilidade', fontsize=10)
    ax.set_title(f'✓ Correto: {y_labels[y_test[idx]]}', fontsize=11, fontweight='bold', color='green')
    ax.grid(axis='x', alpha=0.3)

# Incorretas
incorrect_indices = np.where(y_pred != y_test)[0]
for i in range(min(3, len(incorrect_indices))):
    idx = incorrect_indices[i]
    ax = axes[1, i]
    
    # Criar visualização de confiança
    classes = y_labels
    probs = y_pred_proba[idx]
    colors = ['red' if j == y_pred[idx] else ('orange' if j == y_test[idx] else 'lightgray') 
              for j in range(len(classes))]
    ax.barh(classes, probs, color=colors)
    ax.set_xlim([0, 1])
    ax.set_xlabel('Probabilidade', fontsize=10)
    title_text = f'✗ Incorreto\nReal: {y_labels[y_test[idx]]}, Predito: {y_labels[y_pred[idx]]}'
    ax.set_title(title_text, fontsize=11, fontweight='bold', color='red')
    ax.grid(axis='x', alpha=0.3)

# Remover subplots vazios se necessário
for i in range(min(3, len(correct_indices)), 3):
    fig.delaxes(axes[0, i])
for i in range(min(3, len(incorrect_indices)), 3):
    fig.delaxes(axes[1, i])

plt.tight_layout()
plt.show()

print("\n✓ Todas as visualizações geradas com sucesso!")

## 10. Resumo e Recomendações

In [ ]:
print("\n" + "="*80)
print("RESUMO DO MODELO E RECOMENDAÇÕES")
print("="*80)

print("\n📊 RESUMO GERAL:")
print(f"""
  • Modelo: CNN 1D com 4 camadas convolucionais paralelas
  • Tamanho vocabulário: {VOCAB_SIZE}
  • Dimensão embedding: {EMBEDDING_DIM}
  • Comprimento máximo sequência: {MAX_SEQUENCE_LENGTH}
  • Total de parâmetros: {model.count_params():,}
  • Épocas treinadas: {len(history.history['loss'])}
""")

print("📈 DESEMPENHO NO CONJUNTO DE TESTE:")
print(f"""
  • Acurácia: {test_accuracy:.4f}
  • Precisão: {test_precision:.4f}
  • Recall: {test_recall:.4f}
  • F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}
""")

print("🎯 RECOMENDAÇÕES PARA MELHORAR O MODELO:")
recommendations = [
    "1. Aumentar tamanho do dataset - mais dados melhoram generalização",
    "2. Tentar arquitetura Transformer/BERT - melhor compreensão contextual",
    "3. Implementar class weights - lidar com desbalanceamento de classes",
    "4. Fazer feature engineering - incluir metadados (data, área jurídica, etc)",
    "5. Usar word embeddings pré-treinados (Word2Vec, GloVe, FastText)",
    "6. Testar diferentes tamanhos de embedding e filtros",
    "7. Aumentar dropouts se houver sinais de overfitting",
    "8. Fazer ablação de arquitetura - remover/adicionar camadas",
    "9. Implementar ensemble com modelos complementares",
    "10. Validação cruzada estratificada para estimativa mais robusta"
]

for rec in recommendations:
    print(f"  {rec}")

print("\n💾 SALVANDO MODELO:")
# Salvar modelo
model_path = '/Users/bruno.silva/Projects/doutorado/cnn-classificador-peticoes/modelo_cnn_desfecho.h5'
model.save(model_path)
print(f"  ✓ Modelo salvo em: {model_path}")

# Salvar tokenizer
import pickle
tokenizer_path = '/Users/bruno.silva/Projects/doutorado/cnn-classificador-peticoes/tokenizer_desfecho.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)
print(f"  ✓ Tokenizer salvo em: {tokenizer_path}")

# Salvar label encoder
encoder_path = '/Users/bruno.silva/Projects/doutorado/cnn-classificador-peticoes/label_encoder_desfecho.pkl'
with open(encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)
print(f"  ✓ Label Encoder salvo em: {encoder_path}")

print("\n✅ PROTOTIPAGEM CONCLUÍDA COM SUCESSO!")
print("\nPróximos passos:")
print("  1. Refinar modelo com dados reais")
print("  2. Explorar arquiteturas mais avançadas (Transformers)")
print("  3. Implementar API de predição em produção")
print("  4. Monitorar performance em ambiente de produção")